In [4]:
import datetime
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

In [71]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    start="2017-01-01",
    end="2023-01-01",
    tickers="NDX",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [72]:
yfdata = response.copy().dropna()
yfdata = yfdata[yfdata.Volume != 0]
# yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,4900.85,4928.49,4884.52,4911.33,4911.33,1886200000
2017-01-04,4920.79,4944.74,4919.80,4937.21,4937.21,1883360000
2017-01-05,4936.35,4967.90,4935.34,4964.95,4964.95,1792610000
2017-01-06,4973.87,5020.70,4957.82,5007.08,5007.08,1710770000
2017-01-09,5013.82,5033.32,5009.45,5024.90,5024.90,1885500000
...,...,...,...,...,...,...
2022-10-31,11465.21,11482.99,11331.26,11405.57,11405.57,4753740000
2022-11-01,11571.53,11574.39,11278.28,11288.95,11288.95,4677520000
2022-11-02,11300.27,11410.91,10903.48,10906.34,10906.34,5436420000


In [73]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]

In [74]:
class My_Strategy(Strategy):
    # RSI
    prop_rsi = 49  # 14
    prop_rsi_high = 60  # 70
    prop_rsi_low = 50  # 30

    take_profit = 4
    stop_loss = 6

    def init(self):
        self.macd, self.macd_signal = self.I(MACD, self.data.df)
        self.rsi = self.I(RSI, self.data.df, self.prop_rsi)

    def not_trade_with_rsi_range(self):
        return self.prop_rsi_low < self.rsi[-1] and self.rsi[-1] < self.prop_rsi_high

    def golden_cross_with_macd_day(self):
        return crossover(self.macd, self.macd_signal)

    def dead_cross_with_macd_day(self):
        return crossover(self.macd_signal, self.macd)

    def next(self):
        # 計算できていない場合トレードしない
        if len(self.data.index) < 21:  # MACD
            return

        if self.not_trade_with_rsi_range():
            return

        # MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_macd_day():
            if self.position.is_short:
                self.position.close()

            if self.position.is_long:
                return

            self.buy(
                sl=self.data.Close[-1] * (1 - (self.stop_loss / 100)),
                tp=self.data.Close[-1] * (1 + (self.take_profit / 100)),
            )
            return

        # MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_macd_day():
            if self.position.is_long:
                self.position.close()

            if self.position.is_short:
                return

            self.sell(
                sl=self.data.Close[-1] * (1 + (self.stop_loss / 100)),
                tp=self.data.Close[-1] * (1 - (self.take_profit / 100)),
            )
            return


bt = Backtest(
    yfdata, My_Strategy, cash=10000000, commission=0.003, exclusive_orders=True
)


# 最適化
optimize = bt.optimize(
    # prop_rsi=range(7, 21, 7),
    prop_rsi_high=range(30, 70, 10),
    prop_rsi_low=range(30, 70, 10),
    # take_profit=range(2, 8, 2),
    # stop_loss=range(2, 8, 2),
    constraint=lambda p: p.prop_rsi_low < p.prop_rsi_high,
    maximize="SQN",
)
bt.plot()
print(optimize)
print(optimize._strategy)


# optimize = bt.optimize(
#     # prop_rsi=range(7, 21, 7),
#     prop_rsi_high=50,
#     prop_rsi_low=30,
#     # take_profit=range(2, 8, 2),
#     # stop_loss=range(2, 8, 2),
#     constraint=lambda p: p.prop_rsi_low < p.prop_rsi_high,
#     maximize="SQN",
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)

# 出力
# output = bt.run()
# print("Return : " + str(output["Return [%]"]) + "%")
# print("Trades : " + str(output["# Trades"]))
# print(output)
# bt.plot()

Start                     2017-01-03 00:00:00
End                       2022-11-04 00:00:00
Duration                   2131 days 00:00:00
Exposure Time [%]                       24.86
Equity Final [$]                  10131887.02
Equity Peak [$]                   10131887.02
Return [%]                               1.32
Buy & Hold Return [%]                  121.06
Return (Ann.) [%]                        0.22
Volatility (Ann.) [%]                    6.95
Sharpe Ratio                             0.03
Sortino Ratio                            0.04
Calmar Ratio                             0.01
Max. Drawdown [%]                      -19.28
Avg. Drawdown [%]                       -6.79
Max. Drawdown Duration     1638 days 00:00:00
Avg. Drawdown Duration      424 days 00:00:00
# Trades                                   24
Win Rate [%]                            50.00
Best Trade [%]                           4.29
Worst Trade [%]                         -7.53
Avg. Trade [%]                    

In [7]:
# write HTML file
filename = "Return_" + str(round(output["Return [%]"])) + "%"
bt.plot(filename=filename)


add_html(
    filename + ".html",
    pd.DataFrame(
        [
            {
                "rsi": My_Strategy.prop_rsi,
                "rsi_high": My_Strategy.prop_rsi_high,
                "prop_rsi_low": My_Strategy.prop_rsi_low,
                "memo": "週足RSIと日足MACDを組み合わせた売買方法<br>RSIでMACDのダマシを極力除外する<br>具体的にはRSIがhighとlowの間は購入しない",
            }
        ],
        index=["value"],
    )
    .transpose()
    .to_html(escape=False),
    True,
)
add_html(filename + ".html", pd.DataFrame(output).to_html(), False)

/tmp/ipykernel_111/23335848.py:3: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(filename=filename)
